# Wider Face: Process Training Set

## Description

In this notebook, we'll be developing a way to get face pictures solely based on the Wider Face training set. In order to do this, we've taken some of the pictures in the wider face model and created what we called a positive training set, showing only pictures of the face, and negative training set, showing pictures that don't have a face. We're going to ignore the images with height or width below 16 pixeis. All of the images will then be converted to grayscale for better results.

In [3]:
import cv2
import numpy as np
from random import randint

In [4]:
def crop_image(image, start_x, start_y, height, width):
    return image[start_y: start_y+height, start_x: start_x+width]

In [37]:
def create_positive_set(file_path, image_set_path, save_path, limit):
    count = 0
    with open(file_path, "r") as file_reader:
        while True:
            image_path = file_reader.readline().rstrip()
            img = cv2.imread(image_set_path + image_path)
            num_faces = int(file_reader.readline())
            if num_faces == 0:
                #it's a negative image
                file_reader.readline()
                continue
            for face in range(num_faces):
                image_data = [ int(elem) for elem in file_reader.readline().split()]
                x,y,w,h= image_data[:4]
                if h<32 or w < 32:
                    continue
                cv2.imwrite(save_path + "Image"+str(count)+".jpg", crop_image(img, x, y, h, w))
                count += 1
                if count > limit:
                    return 

In [14]:
path = "./Image Resources/Dataset/"

In [39]:
images_path = path + "WIDER_train/images/"
file_path = path + "wider_face_split/wider_face_train_bbx_gt.txt"
positive_path = path + "WIDER_train/cropped images/positive/"
train = create_positive_set(file_path, images_path, positive_path, 10000)

In [40]:
images_path = path + "WIDER_val/images/"
file_path = path + "wider_face_split/wider_face_val_bbx_gt.txt"
positive_path = path + "WIDER_val/cropped images/positive/"
train = create_positive_set(file_path, images_path, positive_path, 2000)

In [6]:
def intersect_images(images_to_avoid, start_x, start_y, width, height):
    end_x = start_x + width
    end_y = start_y + height
    for avoid in images_to_avoid:
        center_avoid_x = avoid[0] + avoid[3]//2
        center_avoid_y = avoid[1] + avoid[2]//2
        if center_avoid_x > start_x and center_avoid_x < end_x \
        and center_avoid_y > start_y and center_avoid_y < end_y:
            return True
    center_x = start_x + width//2
    center_y = start_y + height//2
    for avoid in images_to_avoid:
        end_avoid_x = avoid[0] + avoid[3]
        end_avoid_y = avoid[1] + avoid[2]
        if center_x > avoid[0] and center_x < end_avoid_x \
        and center_y > avoid[0] and center_y < end_avoid_y:
            return True
    return False

In [22]:
def create_negative_set(file_path, image_set_path, save_path, limit):
    count = 0
    with open(file_path, "r") as file_reader:
        while True:
            image_path = file_reader.readline().rstrip()
            img = cv2.imread(image_set_path + image_path)
            try:
                img_height, img_width, _ = img.shape
            except Exception as e:
                print(image_path)
                return
            num_faces = int(file_reader.readline().rstrip())
            #List of images from which I have to avoid
            images_to_avoid = []
            if num_faces > 5:
                for face in range(num_faces):
                    file_reader.readline()
                continue
            else:
                if num_faces != 0:
                    for face in range(num_faces):
                        images_to_avoid.append([int(elem) for elem in file_reader.readline().split()[:4]])
                else:
                    file_reader.readline().split()
            #Going to create 5 images per photo, to force more diversity between photos
            for new_img in range(5):
                h = randint(32, 64)
                w = randint(32, 64)
                while True:
                    x = randint(0, img_width - w - 1)
                    y = randint(0, img_height - h - 1)
                    if not intersect_images(images_to_avoid, x, y, h, w):
                        break
                cv2.imwrite(save_path + "Image"+str(count)+".jpg", crop_image(img, x, y, h, w))
                count += 1
                if count > limit:
                    return 
            

In [ ]:
images_path = path + "WIDER_train/images/"
file_path = path + "wider_face_split/wider_face_train_bbx_gt.txt"
negative_path = path + "WIDER_train/cropped images/negative/"
create_negative_set(file_path, images_path, negative_path, 10000)

In [ ]:
images_path = path + "WIDER_val/images/"
file_path = path + "wider_face_split/wider_face_val_bbx_gt.txt"
negative_path = path + "WIDER_val/cropped images/negative/"
create_negative_set(file_path, images_path, negative_path, 2000)